# Validation notebook for running in Google Colab


## Imports and function definitions

In [ ]:
# If you're using Google Colab and not running locally, run this cell.

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

# Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
from google.colab import drive
import glob
import subprocess
from pydub import AudioSegment
import os
import nemo.collections.asr.models as nemo_asr_models

In [ ]:
# If using Google Colab, mount your Google Drive to access the data
drive.mount('/content/drive')

In [ ]:
def convert_audio_file(input_file, output_file):
    # Load the audio file
    audio = AudioSegment.from_file(input_file)

    # Set the desired sample rate and channels
    target_sample_rate = 16000
    target_channels = 1

    # Resample and set channels
    audio = audio.set_frame_rate(target_sample_rate).set_channels(target_channels)

    # Export the result as WAV
    audio.export(output_file, format="wav")

def convert_volunteer_files():
  NEMO_ROOT = os.getcwd()
  data_dir = os.path.join(NEMO_ROOT,'data/volunteer_files_wav')
  os.makedirs(data_dir, exist_ok=True)

  # Get the path to the folder containing the audio files
  audio_folder_path = './data/volunteer_files'

  # Get a list of all the audio files in the folder
  audio_files = glob.glob(audio_folder_path + '/*')

  # Read each audio file
  for audio_file_path in audio_files:
    file_name = audio_file_path[len(audio_folder_path):]
    print('Converting: ' + file_name)
    wav_path = data_dir + file_name[:-4] + '.wav'
    convert_audio_file(audio_file_path, wav_path)

  print("Finished conversion.\n******")

def convert_validation_files():
  NEMO_ROOT = os.getcwd()
  data_dir = os.path.join(NEMO_ROOT,'data/validation_files_wav')
  os.makedirs(data_dir, exist_ok=True)

  # Get the path to the folder containing the audio files
  audio_folder_path = './data/validation_files'

  # Get a list of all the audio files in the folder
  audio_files = glob.glob(audio_folder_path + '/*')

  # Read each audio file
  for audio_file_path in audio_files:
    file_name = audio_file_path[len(audio_folder_path):]
    print('Converting: ' + file_name)
    wav_path = data_dir + file_name[:-4] + '.wav'
    convert_audio_file(audio_file_path, wav_path)

  print("Finished conversion.\n******")

In [ ]:
"""
Speaker Verification to verify the audio files  

Input: 
- audio files should be 16KHz mono channel wav files

Output:
- True: if two provided audio files are from the same speaker 
- False: otherwise
"""
def speaker_recognition(file_path_1, file_path_2):
  speaker_model = nemo_asr_models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
  decision = speaker_model.verify_speakers(file_path_1, file_path_2)
  print(decision)
  return decision


## Validation

In [ ]:
# Step 1 - convert validation files

# Step 2 - iterate through all wav files and call speaker_recognition
# Step 2.a iterate
# Step 2.b match names of files to get actual value
# Step 2.c call speaker_recognition to get predicted value
# Step 2.d print results and write to excel

# Step 3 - analyzing magic